In [127]:
import tfFLAGS
tfFLAGS.LEARNING_RATE_DECAY_FACTOR = 0.1;           tfFLAGS.NUM_CLASSES = 10;       tfFLAGS.run_once = True
tfFLAGS.NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 50000;   tfFLAGS.batch_size = 128;       tfFLAGS.use_fp16 = False
tfFLAGS.NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = 10000;    tfFLAGS.max_steps = 4000;      tfFLAGS.log_frequency = 100
tfFLAGS.MOVING_AVERAGE_DECAY = 0.9999;   tfFLAGS.NUM_EPOCHS_PER_DECAY = 350.0;      tfFLAGS.IMAGE_SIZE = 32;
tfFLAGS.INITIAL_LEARNING_RATE = 0.1;     tfFLAGS.eval_interval_secs = 60 * 5;       tfFLAGS.num_examples = 1000
num_gpus = 2

In [128]:
conv1Size = 2; conv1Out = 4; conv1Shape = [conv1Size, conv1Size, 3, conv1Out]
conv2Size = 2; conv2Shape = [conv2Size, conv2Size, conv1Out, 10]

pool1S = 2; pool1ksize=[1, pool1S, pool1S, 1]; pool1St = 2; pool1strides=[1, pool1St, pool1St, 1]; pool1padding='SAME'
pool2S = 2; pool2ksize=[1, pool2S, pool2S, 1]; pool2St = 2; pool2strides=[1, pool2St, pool2St, 1]; pool2padding='SAME'

local3InputDepth = 64 * conv2Shape[-1]; local3OutputDepth = local3InputDepth
local4InputDepth = local3OutputDepth; local4OutputDepth = 64
softmax_linearInput = local4OutputDepth

In [129]:
import tensorflow as tf

In [130]:
from MyModel import * 
def inferenceOnJupyter(images):
    # conv1
    with tf.variable_scope('conv1') as scope:
        kernel = variable_with_weight_decay('weights', shape=conv1Shape, stddev=5e-2, wd=0.0)
        conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
        biases = variable_on_cpu('biases', [conv1Shape[-1]], tf.constant_initializer(0.0))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name=scope.name)
        activation_summary(conv1)
    # pool1
    pool1 = tf.nn.max_pool(conv1, ksize=pool1ksize, strides=pool1strides, padding=pool1padding, name='pool1')
    # norm1
    norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm1')

    # conv2
    with tf.variable_scope('conv2') as scope:
        kernel = variable_with_weight_decay('weights', shape=conv2Shape, stddev=5e-2, wd=0.0)
        conv = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = variable_on_cpu('biases', [conv2Shape[-1]], tf.constant_initializer(0.1))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name=scope.name)
        activation_summary(conv2)

    # norm2
    norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm2')
    # pool2
    pool2 = tf.nn.max_pool(norm2, ksize=pool2ksize, strides=pool2strides, padding=pool2padding, name='pool2')

    # local3
    with tf.variable_scope('local3') as scope:
        # Move everything into depth so we can perform a single matrix multiply.
        reshape = tf.reshape(pool2, [tfFLAGS.batch_size, -1])
        local3InputDepth = reshape.get_shape()[1].value
        print_('local3InputDepth:', local3InputDepth, ' | local3OutputDepth:', local3OutputDepth, reshape.get_shape())
        weights = variable_with_weight_decay('weights', shape=[local3InputDepth, local3OutputDepth], stddev=0.04, wd=0.004)
        biases = variable_on_cpu('biases', [local3OutputDepth], tf.constant_initializer(0.1))
        local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)        
        activation_summary(local3)

    # local4
    with tf.variable_scope('local4') as scope:
        weights = variable_with_weight_decay('weights', shape=[local3OutputDepth, local4OutputDepth], stddev=0.04, wd=0.004)
        biases = variable_on_cpu('biases', [local4OutputDepth], tf.constant_initializer(0.1))
        local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name=scope.name)
        activation_summary(local4)
        
    with tf.variable_scope('softmax_linear') as scope:
        weights = variable_with_weight_decay('weights', [softmax_linearInput, tfFLAGS.NUM_CLASSES], stddev=1/192.0, wd=0.0)
        biases = variable_on_cpu('biases', [tfFLAGS.NUM_CLASSES], tf.constant_initializer(0.0))
        softmax_linear = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
        activation_summary(softmax_linear)
    return softmax_linear

In [131]:
from PythonVersionHandler import print_
print_('Network summary:')
print_('conv1Shape:', conv1Shape)
print_('pool1ksize:', pool1ksize, ' | pool1strides:', pool1strides, ' | pool1padding:', pool1padding)
print_('conv2Shape:', conv2Shape)
print_('pool2ksize:', pool2ksize, ' | pool2strides:', pool2strides, ' | pool2padding:', pool2padding)
print_('local3InputDepth:', local3InputDepth, ' | local3OutputDepth:', local3OutputDepth)
print_('local4InputDepth:', local4InputDepth, ' | local4OutputDepth:', local4OutputDepth)
print_('softmax_linearInput:', softmax_linearInput)
print_()
numParamConv1 = (conv1Shape[0] * conv1Shape[1] * conv1Shape[2]) * conv1Shape[-1]
print_('Number of hidden parameters of conv1:', numParamConv1)
print_('Number of hidden parameters of conv1Biases:', conv1Shape[-1]) 
numParamConv1 += conv1Shape[-1]
#print_('Number of hidden parameters of norm1:', numParamConv1 / pool1S**1)
numParamConv2 = (conv2Shape[0] * conv2Shape[1] * conv2Shape[2]) * conv2Shape[-1]
print_('Number of hidden parameters of conv2:', numParamConv2)
print_('Number of hidden parameters of conv2Biases:', conv2Shape[-1]) 
numParamConv2 += conv2Shape[-1]
numParamLocal3 = (local3InputDepth) * local3OutputDepth
print_('Number of hidden parameters of local3:', numParamLocal3)
print_('Number of hidden parameters of local3Biases:', local3OutputDepth)
numParamLocal3 += local3OutputDepth
numParamLocal4 = (local4InputDepth) * local4OutputDepth
print_('Number of hidden parameters of local4:', numParamLocal4)
print_('Number of hidden parameters of local4Biases:', local4OutputDepth)
numParamLocal4 += local4OutputDepth
numParamsoftmax = (softmax_linearInput) * tfFLAGS.NUM_CLASSES
print_('Number of hidden parameters of softmax:', numParamsoftmax)
print_('Number of hidden parameters of softmaxBiases:', tfFLAGS.NUM_CLASSES)
numParamsoftmax += tfFLAGS.NUM_CLASSES
print_('Total number of hidden parameters:', numParamConv1 + numParamConv2 + numParamLocal3 + numParamLocal4 + numParamsoftmax)

Network summary:
conv1Shape: [2, 2, 3, 4]
pool1ksize: [1, 2, 2, 1]  | pool1strides: [1, 2, 2, 1]  | pool1padding: SAME
conv2Shape: [2, 2, 4, 10]
pool2ksize: [1, 2, 2, 1]  | pool2strides: [1, 2, 2, 1]  | pool2padding: SAME
local3InputDepth: 640  | local3OutputDepth: 640
local4InputDepth: 640  | local4OutputDepth: 64
softmax_linearInput: 64

Number of hidden parameters of conv1: 48
Number of hidden parameters of conv1Biases: 4
Number of hidden parameters of conv2: 160
Number of hidden parameters of conv2Biases: 10
Number of hidden parameters of local3: 409600
Number of hidden parameters of local3Biases: 640
Number of hidden parameters of local4: 40960
Number of hidden parameters of local4Biases: 64
Number of hidden parameters of softmax: 640
Number of hidden parameters of softmaxBiases: 10
Total number of hidden parameters: 452136


In [132]:
import MyModel
inferenceOnMyModel = MyModel.inference
MyModel.inference = inferenceOnJupyter

In [133]:
from runCIFAR10_Tensorflow import main
main()

########################################################################################
########################################################################################
########################################################################################
2017-05-06 07:36:16.559719: Running on server...
The experiment details:
max_steps = 4000 log_frequency = 100 num_gpus = 2
Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.
local3InputDepth: 640  | local3OutputDepth: 640 (128, 640)
Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.
local3InputDepth: 640  | local3OutputDepth: 640 (128, 640)
48
4
160
10
409600
640
40960
64
640
10
total_parameters 452136
2017-05-06 07:36:38.020791: step 0, loss = 3.42 (16.3 examples/sec; 7.838 sec/batch)
2017-05-06 07:36:50.364378: step 100, loss = 3.33 (2326.5 examples/sec; 0.055 sec/batch)
2017-05-06 07:37:01.486676: step 200, loss = 3.25 (2452.4 examples/